# Cleaning and Merging Both Tables

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime, math

In [ ]:
%%capture
mta = pd.read_csv('../../data/mta_1708.csv', error_bad_lines=False)
# mta.set_index('PublishedLineName', inplace=True)
m100 = mta.loc[(mta['PublishedLineName']== 'M100') & (mta['DestinationName'] == 'INWOOD 220 ST via AMSTERDAM via BWAY'),]

In [ ]:
arrivals = m100.loc[m100['ArrivalProximityText']=='at stop']
arrivals.head(20)

In [ ]:
new = arrivals.loc[:,['RecordedAtTime','VehicleRef']].sort_values(['VehicleRef', 'RecordedAtTime'], ascending=True)
uniqueRef = new['VehicleRef'].unique()
new.RecordedAtTime = pd.to_datetime(new.RecordedAtTime)

## Setting the time deltas only within the same vehicle refs
https://stackoverflow.com/questions/20648346/computing-diffs-within-groups-of-a-dataframe


In [ ]:
new.sort_values(['VehicleRef', 'RecordedAtTime'], inplace=True)
new['diffs'] = new['RecordedAtTime'].diff()
mask = new.VehicleRef != new.VehicleRef.shift(1)
new['diffs'][mask] = np.nan
new

In [ ]:
new.dropna()
new = new[new.diffs > '02:00:00']
new

mta.describe()

# Saving our Progress/Merging Weather Data

In [2]:
%%capture
weather = pd.read_csv('../../data/1401011_weather_data.csv', error_bad_lines=False)

In [3]:
# weather.tail()

In [4]:
%%capture
m100 = pd.read_csv('../../data/busBoarding.csv', error_bad_lines=False)

In [5]:
m100.tail()

,Unnamed: 0,Date,passengerId,passengerArrivalTime,BusDepartureTime,VehicleRef,TimeDelta,numPassengersPerBus
14495,14495,2017-08-29 00:00:00.000000000,225_evening,2017-08-29 19:27:32.114466781,2017-08-29 19:32:56.000000000,NYCT_8363,0 days 00:09:58.000000000,5
14496,14496,2017-08-29 00:00:00.000000000,228_evening,2017-08-29 19:32:43.174941548,2017-08-29 19:32:56.000000000,NYCT_8363,0 days 00:09:58.000000000,5
14497,14497,2017-08-29 00:00:00.000000000,267_evening,2017-08-29 19:30:06.568012301,2017-08-29 19:32:56.000000000,NYCT_8363,0 days 00:09:58.000000000,5
14498,14498,2017-08-29 00:00:00.000000000,440_evening,2017-08-29 19:42:07.881809817,2017-08-29 19:42:32.000000000,NYCT_8375,0 days 00:00:31.000000000,2
14499,14499,2017-08-29 00:00:00.000000000,63_evening,2017-08-29 19:42:12.460652724,2017-08-29 19:42:32.000000000,NYCT_8375,0 days 00:00:31.000000000,2


In [6]:
m100 = m100[['passengerArrivalTime', 'numPassengersPerBus', 'BusDepartureTime']]

In [7]:
m100.head()

,passengerArrivalTime,numPassengersPerBus,BusDepartureTime
0,2017-08-01 00:07:57.749880444,8,2017-08-01 00:11:39.000000000
1,2017-08-01 00:01:01.377135460,8,2017-08-01 00:11:39.000000000
2,2017-08-01 00:09:06.986231475,8,2017-08-01 00:11:39.000000000
3,2017-08-01 00:03:57.560299001,8,2017-08-01 00:11:39.000000000
4,2017-08-01 00:06:26.409442936,8,2017-08-01 00:11:39.000000000


## Data Cleaning
Credit: Angelika

In [8]:
newWeather = weather[['DATE','HOURLYVISIBILITY', 'HOURLYWindSpeed', 'HOURLYPrecip']]

In [9]:
newWeather = newWeather.dropna()
newWeather = newWeather[~newWeather.HOURLYPrecip.str.contains("T")]
newWeather = newWeather[~newWeather.HOURLYPrecip.str.contains("s")]

In [10]:
newWeather = newWeather[~newWeather.HOURLYVISIBILITY.str.contains("V")]
newWeather.HOURLYVISIBILITY=pd.to_numeric(newWeather.HOURLYVISIBILITY)

This is a function that combines the weather and bus tables into one based on the day of the month and the hour of the day. We'll call this function after we've cleaned up the weather tables.

In [11]:
'''
    @params:
        weather: weather table
        bus: bus table
''' 

def mergeOnDateTime(bus, weather):
    weather['DATE'] = pd.to_datetime(weather['DATE'])
    bus['BusDepartureTime'] = pd.to_datetime(bus['BusDepartureTime'])
    
    weather['stringTime'] = weather['DATE'].apply(lambda x: x.strftime('%Y-%m-%d %H'))
    bus['stringTime'] = bus['BusDepartureTime'].apply(lambda x: x.strftime('%Y-%m-%d %H'))
    
    newTable = pd.merge(left=bus, right=weather,  how='inner', on=['stringTime'])
    
    newTable.drop(columns=['stringTime', 'DATE'], inplace=True, axis=1)
    
    return newTable

In [12]:
# Time gate to August

newWeather = newWeather[(newWeather['DATE'] > '2017-08-01') & (newWeather['DATE'] < '2017-09-01')].reset_index().dropna()

We'll enforce the datatype on the Precipitation and Visibility to floats.

In [13]:

# Fix some data types
newWeather['HOURLYPrecip'] = pd.to_numeric(newWeather['HOURLYPrecip'], downcast='float', errors='coerce')
newWeather['HOURLYVISIBILITY'] = pd.to_numeric(newWeather['HOURLYVISIBILITY'], downcast='float', errors='coerce')
# Bound hour of day
# newWeather = newWeather[(newWeather['HOUR'] > 4) & (newWeather['HOUR'] < 20)]
newWeather.reset_index()
newWeather.drop(columns=['index'], inplace=True, axis=1)


In [14]:
newWeather.dtypes
newWeather.shape

(759, 4)

In [15]:
m100.dtypes
m100.shape

(14500, 3)

Using the function above, we merge both the tables into one

In [16]:
df = mergeOnDateTime(m100, newWeather)

In [17]:
df.head()
df.dtypes

passengerArrivalTime            object
numPassengersPerBus              int64
BusDepartureTime        datetime64[ns]
HOURLYVISIBILITY               float32
HOURLYWindSpeed                float64
HOURLYPrecip                   float32
dtype: object

In [20]:
df.sort_values(['passengerArrivalTime'], inplace=True)

Redoing time deltas

In [21]:
df.head()

,passengerArrivalTime,numPassengersPerBus,BusDepartureTime,HOURLYVISIBILITY,HOURLYWindSpeed,HOURLYPrecip
1,2017-08-01 00:01:01.377135460,8,2017-08-01 00:11:39,10.0,3.0,0.0
7,2017-08-01 00:01:10.955742168,8,2017-08-01 00:11:39,10.0,3.0,0.0
6,2017-08-01 00:02:46.347575671,8,2017-08-01 00:11:39,10.0,3.0,0.0
3,2017-08-01 00:03:57.560299001,8,2017-08-01 00:11:39,10.0,3.0,0.0
5,2017-08-01 00:04:47.332565580,8,2017-08-01 00:11:39,10.0,3.0,0.0


In [22]:
df['timeTillNext'] = df['RecordedAtTime'].diff().dt.total_seconds()
df['timeTillNext'] = df['timeTillNext'].shift(-1).apply(lambda x: x/60)
df['timeTillNext'] = pd.to_numeric(df['timeTillNext'])

KeyError: 'RecordedAtTime'

In [23]:
%%capture

df.drop(columns='Unnamed: 0', inplace=True, axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [24]:
df.reset_index()


,index,passengerArrivalTime,numPassengersPerBus,BusDepartureTime,HOURLYVISIBILITY,HOURLYWindSpeed,HOURLYPrecip
0,1,2017-08-01 00:01:01.377135460,8,2017-08-01 00:11:39,10.0,3.0,0.00
1,7,2017-08-01 00:01:10.955742168,8,2017-08-01 00:11:39,10.0,3.0,0.00
2,6,2017-08-01 00:02:46.347575671,8,2017-08-01 00:11:39,10.0,3.0,0.00
3,3,2017-08-01 00:03:57.560299001,8,2017-08-01 00:11:39,10.0,3.0,0.00
4,5,2017-08-01 00:04:47.332565580,8,2017-08-01 00:11:39,10.0,3.0,0.00
5,4,2017-08-01 00:06:26.409442936,8,2017-08-01 00:11:39,10.0,3.0,0.00
6,0,2017-08-01 00:07:57.749880444,8,2017-08-01 00:11:39,10.0,3.0,0.00
7,2,2017-08-01 00:09:06.986231475,8,2017-08-01 00:11:39,10.0,3.0,0.00
8,8,2017-08-01 00:13:52.841268012,2,2017-08-01 00:21:06,10.0,3.0,0.00
9,9,2017-08-01 00:15:41.986226221,2,2017-08-01 00:21:06,10.0,3.0,0.00


In [25]:
%%capture
df.drop(columns=['index'], inplace=True, axis=1)

KeyError: "['index'] not found in axis"

In [ ]:
df.drop(columns=['timeDelta'], inplace=True, axis=1)
df.timeDelta.dropna(how='any')

In [26]:
df.drop_duplicates()
df.shape

(14642, 6)

In [27]:
df['ArrivalHour'] = pd.to_datetime(df['passengerArrivalTime']).dt.hour
df['ArrivalSeconds'] = pd.to_datetime(df['passengerArrivalTime']).dt.second
df['ArrivalMinutes'] = pd.to_datetime(df['passengerArrivalTime']).dt.minute


In [28]:
df.tail()

,passengerArrivalTime,numPassengersPerBus,BusDepartureTime,HOURLYVISIBILITY,HOURLYWindSpeed,HOURLYPrecip,ArrivalHour,ArrivalSeconds,ArrivalMinutes
14639,2017-08-29 19:30:06.568012301,5,2017-08-29 19:32:56,8.0,8.0,0.02,19,6,30
14636,2017-08-29 19:30:59.266582854,5,2017-08-29 19:32:56,8.0,8.0,0.02,19,59,30
14638,2017-08-29 19:32:43.174941548,5,2017-08-29 19:32:56,8.0,8.0,0.02,19,43,32
14640,2017-08-29 19:42:07.881809817,2,2017-08-29 19:42:32,8.0,8.0,0.02,19,7,42
14641,2017-08-29 19:42:12.460652724,2,2017-08-29 19:42:32,8.0,8.0,0.02,19,12,42


Now we can see that the rows have a corresponding `timeDelta` in which we calculate the next bus arrival in minutes for the next arrival.

### Saving our Model

In [29]:
df.to_csv('../../data/Merged_Bus_Weather.csv')